In [3]:
import pandas
import implicit

import scipy as sp
import numpy as np
import turicreate as tc

ModuleNotFoundError: No module named 'implicit'

In [3]:
df = pandas.read_csv("user_item_test.csv")
df

,user_id,item_id,user,item,rating,rating2
0,0,0,amit,seahawks,3,1
1,0,1,amit,sf giants,3,1
2,0,2,amit,warriors,3,1
3,1,3,aryana,patriots,3,1
4,1,2,aryana,warriors,5,1
5,2,4,mital,steelers,5,1
6,2,5,mital,penguins,4,1
7,2,6,mital,pirates,3,1
8,2,7,mital,celtics,1,1
9,3,4,kaiden,steelers,5,1


In [4]:
# just the matrix
df_pivot = df.pivot_table(values='rating', index='item', columns='user', aggfunc=len, fill_value=0)
item_user_data = df_pivot.values

sdata = sp.sparse.csr_matrix(item_user_data)
print (sdata)


  (0, 4)	1
  (1, 2)	1
  (2, 4)	1
  (3, 4)	1
  (4, 0)	1
  (5, 0)	1
  (6, 3)	1
  (6, 4)	1
  (7, 0)	1
  (7, 1)	1
  (7, 2)	1


In [5]:
df.pivot_table(values='rating', index='item', columns='user', aggfunc=len, fill_value=0)

user,amit,anika,aryana,kaiden,mital
item,,,,,
celtics,0,0,0,0,1
patriots,0,0,1,0,0
penguins,0,0,0,0,1
pirates,0,0,0,0,1
seahawks,1,0,0,0,0
sf giants,1,0,0,0,0
steelers,0,0,0,1,1
warriors,1,1,1,0,0


## try implicit package


In [6]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sdata)


In [7]:
def recs(recommendations, df, userid):
    user = df.columns[userid]
    
    print("recomemndations for %d: %s" % ( userid, user)) 
    for rr in recommendations:
        item = df.index[rr[0]]
        score = rr[1]
        print (item, score )
    

In [14]:
userid = 5


# exclude what you already like
col = df_pivot.columns[userid]
print(userid, df_pivot.columns[userid])
user_filter  = (df_pivot[col] > 0)
prev_items = df_pivot[user_filter].index
df_items = pandas.DataFrame(df_pivot.index)

item_filter = df_items.isin(prev_items)
exclude_items = df_items[item_filter].dropna().index.values
list(exclude_items)

IndexError: index 5 is out of bounds for axis 0 with size 5

In [13]:

# recommend items for a user
user_items = sdata.T

recommendations = model.recommend(userid, user_items, filter_items=list(exclude_items))
print(recommendations)
recs(recommendations, df_pivot, userid)

[(1, 0.0006362498), (7, 0.00053444505), (4, 0.00037611648), (5, 0.00037610345)]
recomemndations for 4: mital
patriots 0.0006362498
warriors 0.00053444505
seahawks 0.00037611648
sf giants 0.00037610345


In [27]:
recommendations

[(1, 0.0048443787),
 (5, 0.0027556634),
 (4, 0.0027556517),
 (3, 3.9462e-05),
 (2, 3.9458275e-05),
 (6, 3.3341348e-05)]

In [31]:
itemid = recommendations[0][0]
?model.explain(userid, user_items, itemid)

## try turi/graphlab package

https://turi.com/

In [15]:
sf = tc.SFrame(df)
sf

user_id,item_id,user,item,rating,rating2
0,0,amit,seahawks,3,1
0,1,amit,sf giants,3,1
0,2,amit,warriors,3,1
1,3,aryana,patriots,3,1
1,2,aryana,warriors,5,1
2,4,mital,steelers,5,1
2,5,mital,penguins,4,1
2,6,mital,pirates,3,1
2,7,mital,celtics,1,1
3,4,kaiden,steelers,5,1


In [22]:
?tc.recommender.create

In [23]:
sf = tc.SFrame(df)
model = tc.recommender.create(sf, 'user_id', 'item_id', "rating")

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 11 observations with 5 users and 8 items.

Data prepared in: 0.011884s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.183774

Final training RMSE: 0.165856

In [24]:
model.recommend([4])

user_id,item_id,score,rank
4,4,3.497411738742481,1
4,5,3.3327711183916433,2
4,7,3.1914383728395803,3
4,6,3.187069374729286,4
4,1,2.8262692649256094,5
4,0,2.8130994398485525,6
4,3,2.8060123879801138,7


In [25]:
model.recommend([3]).join(sf, on=['item_id']).sort('score')

user_id,item_id,score,rank,user_id.1,user,item,rating,rating2
3,3,2.7883501831780775,7,1,aryana,patriots,3,1
3,0,2.878158744085918,6,0,amit,seahawks,3,1
3,1,2.8984638709913595,5,0,amit,sf giants,3,1
3,7,3.0738303471695287,4,2,mital,celtics,1,1
3,6,3.0739666816185816,3,2,mital,pirates,3,1
3,5,3.1678200546990736,2,2,mital,penguins,4,1
3,2,3.354685518551956,1,1,aryana,warriors,5,1
3,2,3.354685518551956,1,0,amit,warriors,3,1
3,2,3.354685518551956,1,4,anika,warriors,5,1


## turi ranking factorization

In [20]:
model = tc.ranking_factorization_recommender.create(sf, 'user_id', 'item_id')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 11 observations with 5 users and 8 items.

Data prepared in: 0.028947s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.07339

Final training Predictive Error: 0.0159875

In [40]:
?model.recommend([1])

In [21]:
model.recommend([4]).join(sf, on=['item_id']).sort('score')

user_id,item_id,score,rank,user_id.1,user,item,rating,rating2
4,4,0.22591595667776812,7,3,kaiden,steelers,5,1
4,4,0.22591595667776812,7,2,mital,steelers,5,1
4,3,0.25365427034859817,6,1,aryana,patriots,3,1
4,5,0.28802916419731456,5,2,mital,penguins,4,1
4,1,0.29043272289191613,4,0,amit,sf giants,3,1
4,7,0.3070217694964245,3,2,mital,celtics,1,1
4,6,0.3141279114158415,2,2,mital,pirates,3,1
4,0,0.3195513994741424,1,0,amit,seahawks,3,1


In [37]:
sf

user_id,item_id,user,item,rating,rating2
0,0,amit,seahawks,3,1
0,1,amit,sf giants,3,1
0,2,amit,warriors,3,1
1,3,aryana,patriots,3,1
1,2,aryana,warriors,5,1
2,4,mital,steelers,5,1
2,5,mital,penguins,4,1
2,6,mital,pirates,3,1
2,7,mital,celtics,1,1
3,4,kaiden,steelers,5,1
